## Imports and Installs

In [2]:
!pip install yfinance

In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

import time
from datetime import date

## Question 1: IPO Filings Web Scraping and Data Processing

**What's the total sum ($m) of 2023 filings that happened on Fridays?**

In [4]:
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

In [24]:
ipos_df = ipo_dfs[0]
ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Filing Date           326 non-null    datetime64[ns]
 1   Symbol                326 non-null    object        
 2   Company Name          326 non-null    object        
 3   Price Range           326 non-null    object        
 4   Shares Offered        253 non-null    float64       
 5   Avg_price             259 non-null    float64       
 6   Shares_offered_value  250 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 18.0+ KB


In [25]:
ipos_df.head()

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.0,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.0,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.0,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN


In [26]:
# convert to datetime
ipos_df['Filing Date'] = pd.to_datetime(ipos_df['Filing Date'], format="%b %d, %Y")

In [11]:
# Problem --> not always the columns are filled
missing_prices_df = ipos_df[ipos_df['Shares Offered'].astype(str).str.find('-') >= 0]
missing_prices_df

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,2024-05-03,TBN,Tamboran Resources Corporation,-,-
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,-
5,2024-04-22,DRJT,Derun Group Inc,$5.00,-
7,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,-
8,2024-04-15,GAUZ,Gauzy Ltd.,-,-
...,...,...,...,...,...
301,2021-03-02,NSAL,"Navios South American Logistics, Inc.",-,-
315,2021-01-19,DECI,"Decipher Biosciences, Inc.",-,-
317,2020-09-08,GMWX,"GameWorks, Inc.",-,-
318,2020-09-04,SCIT,Sancai Holding Group Ltd.,-,-


In [27]:
ipos_df['Shares Offered'] = pd.to_numeric(ipos_df['Shares Offered'], errors='coerce')

In [28]:
import re

def extract_numbers(input_string):
    price_range = re.search(r'^\$([0-9]+\.[0-9]{2}) - \$([0-9]+\.[0-9]{2})$', input_string)
    if price_range:
        min = float(price_range.group(1))
        max = float(price_range.group(2))
        return round((float(min) + float(max)) / 2, 2)
    pr = re.search(r'^\$([0-9]+)\.[0-9]{2}$', input_string)
    if pr:
        return float(input_string[1:])
    return float('nan')

# Examples
examples = ["Y10_M2", "$9.50 - $10.43", "$39.56"]
for example in examples:
    p1 = extract_numbers(example)
    print(f"{example} ==> {p1}")

Y10_M2 ==> nan
$9.50 - $10.43 ==> 9.96
$39.56 ==> 39.56


In [29]:
ipos_df['Avg_price'] = ipos_df['Price Range'].apply(extract_numbers)

In [30]:
ipos_df['Shares_offered_value'] = ipos_df['Avg_price'] * ipos_df['Shares Offered']

In [31]:
# filter ipos by day of week friday and year
friday_ipos = ipos_df[ipos_df['Filing Date'].dt.dayofweek == 4]
friday_ipos

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,5.000000e+07
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
6,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,10.00,2.500000e+08
9,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN,NaN,NaN
...,...,...,...,...,...,...,...
299,2021-03-05,HRDG,Huarui International New Material Limited,$4.00 - $6.00,2400000.0,5.00,1.200000e+07
311,2021-02-19,EOCO,Elliott Opportunity I Corp.,$10.00,100000000.0,10.00,1.000000e+09
314,2021-01-22,TONY,"Tony Fun, Inc.",$7.00 - $7.50,4177500.0,7.25,3.028688e+07
318,2020-09-04,SCIT,Sancai Holding Group Ltd.,-,NaN,NaN,NaN


In [32]:
friday_ipos = friday_ipos[ipos_df['Filing Date'].dt.year == 2023]
sum_ipos_fill = friday_ipos['Shares_offered_value'].sum()
print(f"{sum_ipos_fill:,}")

285,700,000.0


<ipython-input-32-8dc74d05295d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  friday_ipos = friday_ipos[ipos_df['Filing Date'].dt.year == 2023]


## Question 2:  IPOs "Fixed days hold" strategy


**Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?**

In [33]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2023_df = ipo_dfs[0]
ipos_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      154 non-null    object
 1   Symbol        154 non-null    object
 2   Company Name  154 non-null    object
 3   IPO Price     154 non-null    object
 4   Current       154 non-null    object
 5   Return        154 non-null    object
dtypes: object(6)
memory usage: 7.3+ KB


In [35]:
url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_2024_dfs = pd.read_html(response.text)

ipos_2024_df = ipo_2024_dfs[0]
ipos_2024_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   IPO Date      64 non-null     object
 1   Symbol        64 non-null     object
 2   Company Name  64 non-null     object
 3   IPO Price     64 non-null     object
 4   Current       64 non-null     object
 5   Return        64 non-null     object
dtypes: object(6)
memory usage: 3.1+ KB


In [36]:
ipos_2024_df.head()

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,"May 1, 2024",VIK,Viking Holdings Ltd.,$24.00,$28.94,18.04%
1,"Apr 26, 2024",ZONE,"CleanCore Solutions, Inc.",$4.00,$3.11,-21.50%
2,"Apr 25, 2024",RBRK,"Rubrik, Inc.",$32.00,$34.87,1.28%
3,"Apr 25, 2024",LOAR,Loar Holdings Inc.,$28.00,$51.10,82.50%
4,"Apr 25, 2024",MRX,Marex Group plc,$19.00,$19.15,0.84%


In [37]:
# convert IPO Date to date type
ipos_2023_df['IPO Date'] = pd.to_datetime(ipos_2023_df['IPO Date'], format="%b %d, %Y")
ipos_2024_df['IPO Date'] = pd.to_datetime(ipos_2024_df['IPO Date'], format="%b %d, %Y")
# get ipos_2024_df where IPO Date is "< 2024-03-01"
ipos_2024_df = ipos_2024_df[ipos_2024_df['IPO Date'] < pd.to_datetime('2024-03-01')]
ipos_2024_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31 entries, 33 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      31 non-null     datetime64[ns]
 1   Symbol        31 non-null     object        
 2   Company Name  31 non-null     object        
 3   IPO Price     31 non-null     object        
 4   Current       31 non-null     object        
 5   Return        31 non-null     object        
dtypes: datetime64[ns](1), object(5)
memory usage: 1.7+ KB


In [38]:
# delete ipos 2023 row by Symbol == 'RYZB'
ipos_2023_df = ipos_2023_df[ipos_2023_df['Symbol'] != 'RYZB']
ipos_2023_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 153 entries, 0 to 153
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      153 non-null    datetime64[ns]
 1   Symbol        153 non-null    object        
 2   Company Name  153 non-null    object        
 3   IPO Price     153 non-null    object        
 4   Current       153 non-null    object        
 5   Return        153 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 8.4+ KB


In [39]:
# join ipos_2023 and ipos_2024
stacked_ipos_df = pd.concat([ipos_2023_df, ipos_2024_df], ignore_index=True)
stacked_ipos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    object        
 4   Current       184 non-null    object        
 5   Return        184 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 8.8+ KB


In [45]:
ipos_tickers = stacked_ipos_df[['Symbol']]

# change value of a row 'PTHR' to 'PTHRU'
ipos_tickers.loc[ipos_tickers['Symbol'] == 'PTHR', 'Symbol'] = 'PTHRU'
ipos_tickers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Symbol  184 non-null    object
dtypes: object(1)
memory usage: 1.6+ KB


<ipython-input-45-9735ddfc9865>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ipos_tickers.loc[ipos_tickers['Symbol'] == 'PTHR', 'Symbol'] = 'PTHRU'


In [47]:
ipos_tickers_list = ipos_tickers['Symbol'].to_list()

In [48]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ipos_tickers_list):
  print(i, ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period='max',
                     interval = "1d")

  historyPrices['Ticker'] = ticker
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in range(1, 31):
    historyPrices['growth_future_'+str(i)+'d'] = historyPrices['Adj Close'].shift(-i) / historyPrices['Adj Close']


  if stocks_df.empty:
    stocks_df = historyPrices[historyPrices['Date'] == historyPrices['Date'].min()]
  else:
    stocks_df = pd.concat([stocks_df, historyPrices[historyPrices['Date'] == historyPrices['Date'].min()]], ignore_index=True)

0 IROH


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1 LGCB
2 ZKH


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


3 BAYA


...


[*********************100%%**********************]  1 of 1 completed


 SYNX
182 SDHC


[*********************100%%**********************]  1 of 1 completed


183 ROMA


[*********************100%%**********************]  1 of 1 completed


In [49]:
growth_future = stocks_df[['growth_future_1d', 'growth_future_2d', 'growth_future_3d',
       'growth_future_4d', 'growth_future_5d', 'growth_future_6d',
       'growth_future_7d', 'growth_future_8d', 'growth_future_9d',
       'growth_future_10d', 'growth_future_11d', 'growth_future_12d',
       'growth_future_13d', 'growth_future_14d', 'growth_future_15d',
       'growth_future_16d', 'growth_future_17d', 'growth_future_18d',
       'growth_future_28d', 'growth_future_29d', 'growth_future_30d']].describe()

growth_future

,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,growth_future_5d,growth_future_6d,growth_future_7d,growth_future_8d,growth_future_9d,growth_future_10d,...,growth_future_12d,growth_future_13d,growth_future_14d,growth_future_15d,growth_future_16d,growth_future_17d,growth_future_18d,growth_future_28d,growth_future_29d,growth_future_30d
count,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,184.000000,...,184.000000,184.000000,184.000000,184.000000,183.000000,183.000000,183.000000,182.000000,182.000000,182.000000
mean,0.945820,0.937103,0.930279,0.919575,0.913155,0.906276,0.896519,0.896256,0.898742,0.894293,...,0.926943,0.931986,0.923325,0.923139,0.912754,0.904623,0.893866,0.932887,0.932601,0.927785
std,0.170230,0.205919,0.247641,0.263790,0.294112,0.308535,0.297122,0.318985,0.349821,0.363047,...,0.595602,0.541802,0.565505,0.508499,0.435756,0.403297,0.427298,0.783521,0.781665,0.772243
min,0.153569,0.108733,0.086641,0.094257,0.081118,0.087677,0.085261,0.090123,0.095616,0.096997,...,0.078357,0.077321,0.073870,0.064895,0.056610,0.056265,0.056921,0.046945,0.042803,0.040387
25%,0.921392,0.876712,0.848790,0.832387,0.787687,0.770357,0.788063,0.749143,0.732369,0.709589,...,0.685522,0.689244,0.696126,0.702996,0.712059,0.713676,0.666029,0.640811,0.627851,0.623529
50%,1.000000,0.999058,0.997823,0.995105,0.996456,0.971947,0.978291,0.981124,0.990592,0.978063,...,0.987710,0.993697,0.989805,0.979026,0.986914,0.988235,0.960938,0.974952,0.974438,0.977535
75%,1.011792,1.017794,1.009551,1.008924,1.009369,1.007147,1.006871,1.008127,1.011545,1.010670,...,1.016016,1.019795,1.016382,1.016284,1.014669,1.010848,1.014167,1.040245,1.026362,1.023718
max,1.362069,1.464015,2.380000,2.083710,2.262443,2.529870,2.173913,2.359740,2.751948,3.176087,...,6.779545,5.622727,6.406818,4.656818,4.072398,3.212670,3.500000,9.081632,9.265306,9.372449


In [50]:
# get max from growth_future.loc['75%']
p75 = growth_future.loc['75%']
p75.idxmax(), p75.max()

('growth_future_28d', 1.040244654702553)

## Question 3: Is Growth Concentrated in the Largest Stocks?

**Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?**

In [51]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [52]:
import time

stocks_df = pd.DataFrame({'A' : []})

ALL_STOCKS = LARGEST_STOCKS + LARGE_STOCKS

for i,ticker in enumerate(ALL_STOCKS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     start = "2013-12-22",
                     end= "2023-12-31",
                     interval = "1d")

  historyPrices['ticker'] = ticker
  # check if a stock is in largest o large list and set ticker_category
  if ticker in LARGEST_STOCKS:
    historyPrices['ticker_category'] = 'LARGEST'
  else:
    historyPrices['ticker_category'] = 'LARGE'

  historyPrices['date'] = historyPrices.index.date

  # historical returns
  historyPrices['growth_'+str(7)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(7)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed


2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


65 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


66 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [ ]:
stocks_df[['date', 'ticker_category', 'ticker', 'growth_7d']].head()

,date,ticker_category,ticker,growth_7d
0,2013-12-23,LARGEST,MSFT,NaN
1,2013-12-24,LARGEST,MSFT,NaN
2,2013-12-26,LARGEST,MSFT,NaN
3,2013-12-27,LARGEST,MSFT,NaN
4,2013-12-30,LARGEST,MSFT,NaN


In [54]:
stocks_7d_growth = pd.pivot_table(stocks_df,
                       values='growth_7d',
                       index=['date'],
                       columns=['ticker_category'],
                       aggfunc="mean"
)
stocks_7d_growth.head(9)

ticker_category,LARGE,LARGEST
date,,
2014-01-02,0.980160,0.992433
2014-01-03,0.996904,0.994669
2014-01-06,0.995651,0.993711
2014-01-07,0.990978,0.991773
2014-01-08,0.989366,0.987206
2014-01-09,0.991875,0.980598
2014-01-10,0.989286,0.981773
2014-01-13,0.998538,0.992337
2014-01-14,1.008288,0.998351


In [55]:
stocks_7d_growth['LARGE_outperforms_LARGEST'] = np.where(stocks_7d_growth['LARGE'] > stocks_7d_growth['LARGEST'], True, False)
stocks_7d_growth.head(9)

ticker_category,LARGE,LARGEST,LARGE_outperforms_LARGEST
date,,,
2014-01-02,0.980160,0.992433,False
2014-01-03,0.996904,0.994669,True
2014-01-06,0.995651,0.993711,True
2014-01-07,0.990978,0.991773,False
2014-01-08,0.989366,0.987206,True
2014-01-09,0.991875,0.980598,True
2014-01-10,0.989286,0.981773,True
2014-01-13,0.998538,0.992337,True
2014-01-14,1.008288,0.998351,True


In [56]:
# get the porcentage of days where Large outperforms Largest
stocks_7d_growth['LARGE_outperforms_LARGEST'].mean().round(4) * 100


47.07

## Question 4: Trying Another Technical Indicators strategy

**What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?**

In [58]:
!gdown 1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb

Downloading...
From: https://drive.google.com/uc?id=1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb
To: /content/stocks_df_combined_trunc_2014_2023.parquet.brotli
100% 47.3M/47.3M [00:01<00:00, 39.6MB/s]


In [59]:
# pandas read df from downloaded file
df = pd.read_parquet('stocks_df_combined_trunc_2014_2023.parquet.brotli')
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80762 entries, 0 to 80761
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 104.7+ MB


In [60]:
df.columns.values

array(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d',
       'growth_7d', 'growth_30d', 'growth_90d', 'growth_365d',
       'growth_future_5d', 'SMA10', 'SMA20', 'growing_moving_average',
       'high_minus_low_relative', 'volatility',
       'is_positive_growth_5d_future', 'ticker_type', 'index_x', 'adx',
       'adxr', 'apo', 'aroon_1', 'aroon_2', 'aroonosc', 'bop', 'cci',
       'cmo', 'dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
       'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
       'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
       'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
       'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
       'index_y', 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod',
       'ht_dcphase', 'ht_phasor_inphase', 'ht_phasor_quadrature',
       'ht_sine_sine', 'ht_sine_leadsine', 

In [61]:
df.Date.min(), df.Date.max()

(Timestamp('2014-01-01 00:00:00'), Timestamp('2023-12-29 00:00:00'))

In [62]:
df[['Date', 'Ticker', 'Weekday', 'Adj Close_x', 'Adj Close_y', 'cci']]

,Date,Ticker,Weekday,Adj Close_x,Adj Close_y,cci
0,2014-01-02,MSFT,3,31.233059,14.23,57.700615
1,2014-01-03,MSFT,4,31.022930,13.76,1.373763
2,2014-01-06,MSFT,0,30.367352,13.55,-96.631259
3,2014-01-07,MSFT,1,30.602673,12.92,-83.904297
4,2014-01-08,MSFT,2,30.056356,12.87,-147.855135
...,...,...,...,...,...,...
80757,2023-12-22,LT.NS,4,3477.949951,13.03,70.767162
80758,2023-12-26,LT.NS,1,3490.050049,12.99,99.598220
80759,2023-12-27,LT.NS,2,3544.000000,12.43,130.401152
80760,2023-12-28,LT.NS,3,3518.050049,12.47,106.774509


In [67]:
trading_records = pd.DataFrame(columns=['ticker', 'buy_date', 'buy_price', 'sell_date', 'sell_price', 'result', 'cci'])

In [68]:
def buy(df, threshould):
  # get df redords where the cci column is over 200 and day of week = 4, avoid dates < '2023-12-22'
  buy_df = df[(df['cci'] > threshould) & (df['Weekday'] == 4) & (df['Date'] < '2023-12-22')]

  trading_records['ticker'] = buy_df['Ticker']
  trading_records['buy_date'] = buy_df['Date']
  trading_records['buy_price'] = buy_df['Adj Close_x']
  trading_records['cci'] = buy_df['cci']

buy(df, 200)
trading_records

,ticker,buy_date,buy_price,sell_date,sell_price,result,cci
102,MSFT,2014-05-30,34.912762,NaN,NaN,NaN,203.518798
170,MSFT,2014-09-05,39.395618,NaN,NaN,NaN,257.013330
329,MSFT,2015-04-24,41.630741,NaN,NaN,NaN,340.009871
441,MSFT,2015-10-02,40.151123,NaN,NaN,NaN,206.125828
456,MSFT,2015-10-23,46.583046,NaN,NaN,NaN,431.107024
...,...,...,...,...,...,...,...
80491,LT.NS,2022-11-25,2058.108887,NaN,NaN,NaN,210.797547
80638,LT.NS,2023-06-30,2469.979980,NaN,NaN,NaN,219.106421
80653,LT.NS,2023-07-21,2580.430908,NaN,NaN,NaN,359.236523
80687,LT.NS,2023-09-08,2901.600098,NaN,NaN,NaN,303.781710


In [70]:
def sell(df):
  for i, row in trading_records.iterrows():

    sell_row = df.iloc[i+5]
    #print(type(sell_row))
    #print(sell_row)

    trading_records.loc[i, 'sell_date'] = sell_row['Date']
    trading_records.loc[i, 'sell_price'] = sell_row['Adj Close_x']

    result = round((sell_row['Adj Close_x'] - row.buy_price) / row.buy_price * 1000, 2)
    #print(type(result))
    trading_records.loc[i, 'result'] = result

sell(df)

In [73]:
trading_records['result'].sum().round(2)

1069.52